In [1]:
!nvidia-smi

Thu May 30 05:40:14 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Clone threestudio repo

In [2]:
!git clone https://github.com/threestudio-project/threestudio.git

fatal: destination path 'threestudio' already exists and is not an empty directory.


In [3]:
%cd threestudio

/content/threestudio


Install dependencies

In [4]:
!pip install ninja
!pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas networkx pysdf PyMCubes wandb torchmetrics controlnet_aux
!pip install einops kornia taming-transformers-rom1504 git+https://github.com/openai/CLIP.git # zero123
!pip install open3d plotly # mesh visualization

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-njctmclz
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-njctmclz
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


And build some dependencies manually. This may take a while.

In [5]:
!pip install git+https://github.com/ashawkey/envlight.git
!pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
!pip install git+https://github.com/NVlabs/nvdiffrast.git
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

  Cloning https://github.com/ashawkey/envlight.git to /tmp/pip-req-build-tdmbwih4
  Running command git clone --filter=blob:none --quiet https://github.com/ashawkey/envlight.git /tmp/pip-req-build-tdmbwih4
  Resolved https://github.com/ashawkey/envlight.git to commit 05b5851e854429d72ecaf5b206ed64ce55fae677
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/KAIR-BAIR/nerfacc.git (to revision v0.5.2) to /tmp/pip-req-build-k4k5z89q
  Running command git clone --filter=blob:none --quiet https://github.com/KAIR-BAIR/nerfacc.git /tmp/pip-req-build-k4k5z89q
  Running command git checkout -q d84cdf3afd7dcfc42150e0f0506db58a5ce62812
  Resolved https://github.com/KAIR-BAIR/nerfacc.git to commit d84cdf3afd7dcfc42150e0f0506db58a5ce62812
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Created wheel for nerfacc: filename=nerfacc-0.5.2-cp310-cp310-linux_x86_64.whl size=1080729 sha256=f1d2011140dacc24032cbd29cad160e9891c62a985

Login to HuggingFace

In [6]:
from huggingface_hub import interpreter_login

interpreter_login()


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): ··········
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


Now create your own 3D model from text prompts

Here we use the DreamFusion model with DeepFloyd-IF guidance. You may try other models by using different running commands given [here](https://github.com/threestudio-project/threestudio#supported-models).

In [7]:
prompt = "a zoomed out DSLR photo of a baby bunny sitting on top of a stack of pancakes"

In [11]:
!python launch.py --config configs/dreamfusion-if.yaml --train --gpu 0 system.prompt_processor.prompt="$prompt" trainer.max_steps=10000 system.prompt_processor.spawn=false

/usr/local/lib/python3.10/dist-packages/controlnet_aux/mediapipe_face/mediapipe_face_common.py:7: UserWarning: The module 'mediapipe' is not installed. The package will have limited functionality. Please install it using the command: pip install 'mediapipe'
  warnings.warn(
2024-05-30 06:09:25.673954: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-30 06:09:25.674013: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-30 06:09:25.675742: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-30 06:09:25.683980: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Display the rendered video

In [9]:
from IPython.display import HTML
from base64 import b64encode
def display_video(video_path):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=1000 controls>
    <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

In [10]:
# you will see the path to the saving directory at the end of the training logs
# replace save_dir below with that path
save_dir = 'path/to/save/dir'

import os
import glob
video_path = glob.glob(os.path.join(save_dir, "*-test.mp4"))[0]
display_video(video_path)

IndexError: list index out of range

Extract the object mesh.

Here we use an empirical threshold value. You can also first try `system.geometry.isosurface_threshold=auto` and visualize it. Then you can manually adjust the threshold according to the automatically determined value shown in the logs. Increase it if there are too many floaters and decrease it if the geometry is incomplete.


The extraction process takes around 2 mins on T4.

In [ ]:
!python launch.py --config $save_dir/../configs/parsed.yaml --export --gpu 0 resume=$save_dir/../ckpts/last.ckpt system.exporter_type=mesh-exporter system.exporter.context_type=cuda system.geometry.isosurface_threshold=15.0

Visualize the mesh. Or you can directly download the export assets and use them locally.

In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
import glob

mesh_path = glob.glob(os.path.join(save_dir, "*-export/model.obj"))[0]
mesh = o3d.io.read_triangle_mesh(mesh_path)
if not mesh.has_vertex_normals():
  mesh.compute_vertex_normals()
if not mesh.has_triangle_normals():
  mesh.compute_triangle_normals()

triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None
if mesh.has_triangle_normals():
  colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
  colors = tuple(map(tuple, colors))
else:
  colors = (1.0, 0.0, 0.0)
fig = go.Figure(
  data=[
    go.Mesh3d(
      x=vertices[:,0],
      y=vertices[:,1],
      z=vertices[:,2],
      i=triangles[:,0],
      j=triangles[:,1],
      k=triangles[:,2],
      facecolor=colors,
      opacity=0.50)
  ],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
    )
  )
)
fig.show()
